# Introduction to Data Science – Relational Databases
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/* 

Up to now, we've mainly used flat tables to store and process data. Most structured data in the real world, however, is stored in databases, and specifically in [relational databases](https://en.wikipedia.org/wiki/Relational_database). Other database types have specific advantages, such as performance (NoSQL), are suitable for graphs (graph databases, e.g., Neo4j), or are compatibility with in-memory OO data structures (Object Oriented Databases). Relational databases, and their implementations - Relational Database Management Systems (RDBMS), are still the dominant way to store enterprise data though.

Unlike with APIs or web-scarping, it is unlikely that you will be able to access a public database directly. Databases are powerful, but also can be somewhat difficult to use. It's more likely that an API that you interact with is powerd by a relational database behind the scenes, hiding the complexity.

Also, while we've mainly considered reading data, databases are meant also for writing, increasing the potential for abuse. So the most likely scenario for having read/write access to a database is if the database is managed by you or your organization. 

What's the purpose of a relational database? It separates data into multiple tables to avoid redundancy. Here is a simple example for an online transaction in a flat table that describes the data:

| Product Name | Price | Product Description | Customer Name | Customer Address| 
| ------------- | - | - | - | -| 
| MacBook | 2700 | MacBook Pro 15' | Jack Black | San Francisco |  
| Dongle HDMI | 40 | USB-C to HDMI | Jack Black | San Francisco | 
| Dongle USB-B | 40 | USB-C to USB-B | Jack Black | San Francisco | 

The problem here is that we store the name of the customer multiple times, and, if multiple customers purchase the same item, also all the information about the products. Obviously, that's neither great from a storage efficiency perspective nor from an update perspective. If Jack Black wants to update his address, for example, than we'd have to update all three rows. 

A different approach is to separate this table into multiple, meaningful tables. We'll introduce a Customers, Products and a Transactions table. And we'll introduce keys for each row in this table:

**Customers Table**:

| CID | Customer Name | Customer Address | 
| - | - | - |
| 1 | Jack Black | San Francisco |

**Products Table**:

| PID | Product Name | Price | Product Description |
| - | - | - | - | 
| 1 | MacBook | 2700 | MacBook Pro 15' |
| 2 | Dongle HDMI | 40 | USB-C to HDMI |
| 3 | Dongle USB-B | 40 | USB-C to USB-B |

CID and PID are called **primary keys**, as they uniquely identify the row in the table. 

The Transaction Table now refers to the products and the customers only by their keys, which are called **foreign keys** in this context, as they are the primary keys of a "foreign" table.

**Transactions Table**: 

| TID | PID | CID | 
| - | - | - |
| 1 | 1 | 1 | 
| 2 | 3 | 1 | 
| 3 | 2 | 1 | 

Now, if Jack Black wants to update his address, or if the seller wants to update the product description, we can do this in a single place. 

Of course, we have to do a little more work to regenerate (an interesting subset) of transactions, as we had it before. In this lab we'll learn how to do that and more. 

## Relational Database Management System (RDBMS) Transactions

RDBMS are designed to support **read, write, and update** transactions, in addition to operations that **create tables**, etc. In practice, for a database to remain consistent, these transactions have to guarantee [certain properties](https://en.wikipedia.org/wiki/ACID). We'll largely ignore updates, creating tables, etc. – these are operations that you'll likely need if you build a web-application. For the purpose of our data science class, we'll stick to reading/querying for data, as we want to learn **how to get data out of a database**. You can learn more about databases in CS 6530 or the undergrad version CS 5530.

## SQL - The Structured Query Language


[SQL](https://en.wikipedia.org/wiki/SQL) is a domain specific language to execute the transactions described above. We'll look mainly at queries to read and aggregate data. 

Here is a very simple query:

```SQL
SELECT * FROM products
```

This statement selects and retreives all rows (`*`) from the table `products`.

We can restrict the number of lines to retrieve with a `WHERE` clause:

```SQL
SELECT * FROM products WHERE 'Price' > 100
```

This retrieves all the rows where the price is higher than 100. 

We'll go through more specific SQL statements on real examples. You can learn more about SQL [online](https://www.codecademy.com/learn/learn-sql).


## SQLite

Most database management systems are implemented as client-server systems, i.e., the database is hosted on a dedicated server where multiple users can read and write. Examples are [PostgreSQL](https://www.postgresql.org/), [MySQL](https://www.mysql.com/) or [Oracle](https://www.oracle.com/database/index.html). These database management systems are fast and can handle multiple users withouth causing conflicts. However, they require a separate server and installation, so we'll use [SQLite](https://sqlite.org/), a database that works for individual users and stores the whole database in a single file. SQLite is widely used for single-user cases and will serve our purposes. 

While these different databases have different features and performance characteristics, they all support SQL, so the skills you will learn here transfer widely.

We'll also use the [sqlite python interface](https://docs.python.org/3/library/sqlite3.html).

## Sample Database

This tutorial uses the SQLite sample database found [here](http://www.sqlitetutorial.net/sqlite-sample-database/).

Here is a chart of the database schema: 

![](database_schema.png)

 * `employees` table stores employees data such as employee id, last name, first name, etc. It also has a field named `ReportsTo` to specify who reports to whom.
 * `customers` table stores customers data.
 * `invoices & invoice_items` tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.
 * `artists` table stores artists data. It is a simple table that contains only artist id and name.
 * `albums` table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
 * `media_types` table stores media types such as MPEG audio file, ACC audio file, etc.
 * `genres` table stores music types such as rock, jazz, metal, etc.
 * `tracks` table store the data of songs. Each track belongs to one album.
 * `playlists` & `playlist_track` tables: playlists table store data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the playlists table and tracks table is many-to-many. The playlist_track table is used to reflect this relationship.
 
The diagram above highlights the **primary keys** in each table and the relationships, using **foreign keys** to each other table. Note that `employees` has a self-reference, to capture the reports-to relationship.

## Querying the Table

In [1]:
import pandas as pd
import sqlite3 as sq

# we connect to the database, which - in the case of sqlite - is a local file
conn = sq.connect("./chinook.db")
# we retreive the "cursor" on the database.
c = conn.cursor()

# now we can execute a SQL statement
c.execute("SELECT * FROM albums")
# and print the first line from the result
print(c.fetchone())

(1, 'For Those About To Rock We Salute You', 1)


Here, we used a cursor for accessing the data. To retrieve data after executing a SELECT statement, you can either treat the cursor as an iterator, call the cursor’s `fetchone()` method to retrieve a single matching row, or call `fetchall()` to get a list of the matching rows.

This is how you use it as an iterator: 

In [2]:
for row in c:
    print(row)

(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)
(6, 'Jagged Little Pill', 4)
(7, 'Facelift', 5)
(8, 'Warner 25 Anos', 6)
(9, 'Plays Metallica By Four Cellos', 7)
(10, 'Audioslave', 8)
(11, 'Out Of Exile', 8)
(12, 'BackBeat Soundtrack', 9)
(13, 'The Best Of Billy Cobham', 10)
(14, 'Alcohol Fueled Brewtality Live! [Disc 1]', 11)
(15, 'Alcohol Fueled Brewtality Live! [Disc 2]', 11)
(16, 'Black Sabbath', 12)
(17, 'Black Sabbath Vol. 4 (Remaster)', 12)
(18, 'Body Count', 13)
(19, 'Chemical Wedding', 14)
(20, 'The Best Of Buddy Guy - The Millenium Collection', 15)
(21, 'Prenda Minha', 16)
(22, 'Sozinho Remix Ao Vivo', 16)
(23, 'Minha Historia', 17)
(24, 'Afrociberdelia', 18)
(25, 'Da Lama Ao Caos', 18)
(26, 'Acústico MTV [Live]', 19)
(27, 'Cidade Negra - Hits', 19)
(28, 'Na Pista', 20)
(29, 'Axé Bahia 2001', 21)
(30, 'BBC Sessions [Disc 1] [Live]', 22)
(31, 'Bongo Fury', 23)
(32, 'Carnaval 2001', 21)
(33, 'Chill: Brazil (Disc 1)', 24)
(3

And here we retrieve all the data at once:

In [3]:
c.execute("SELECT * FROM albums")
c.fetchall()

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3),
 (6, 'Jagged Little Pill', 4),
 (7, 'Facelift', 5),
 (8, 'Warner 25 Anos', 6),
 (9, 'Plays Metallica By Four Cellos', 7),
 (10, 'Audioslave', 8),
 (11, 'Out Of Exile', 8),
 (12, 'BackBeat Soundtrack', 9),
 (13, 'The Best Of Billy Cobham', 10),
 (14, 'Alcohol Fueled Brewtality Live! [Disc 1]', 11),
 (15, 'Alcohol Fueled Brewtality Live! [Disc 2]', 11),
 (16, 'Black Sabbath', 12),
 (17, 'Black Sabbath Vol. 4 (Remaster)', 12),
 (18, 'Body Count', 13),
 (19, 'Chemical Wedding', 14),
 (20, 'The Best Of Buddy Guy - The Millenium Collection', 15),
 (21, 'Prenda Minha', 16),
 (22, 'Sozinho Remix Ao Vivo', 16),
 (23, 'Minha Historia', 17),
 (24, 'Afrociberdelia', 18),
 (25, 'Da Lama Ao Caos', 18),
 (26, 'Acústico MTV [Live]', 19),
 (27, 'Cidade Negra - Hits', 19),
 (28, 'Na Pista', 20),
 (29, 'Axé Bahia 2001', 21),
 (30, 'BBC Sessions [

## SQL and Pandas

An alternative approach that fits well into our previous workflows is to use pandas to store the table directly, using the [`read_sql()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html) function:

In [4]:
# this is similar to the read_csv()function
pd.read_sql("""SELECT * FROM albums""", conn).head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


This, of course, works only for reading data, but we'll use it for now. 

## Selective Results

When we **specify individual columns** instead of using the `*`, we get only those columns:

In [5]:
pd.read_sql("""SELECT Title FROM albums""", conn).head()

,Title
0,For Those About To Rock We Salute You
1,Balls to the Wall
2,Restless and Wild
3,Let There Be Rock
4,Big Ones


Here, we've used head to display a short list, but we can also do that in the SQL query with the `LIMIT` keyword. Of course, in this case, the data will actually be limited: 

In [6]:
pd.read_sql("""SELECT Title FROM albums LIMIT 3""", conn).head()

,Title
0,For Those About To Rock We Salute You
1,Balls to the Wall
2,Restless and Wild


We can run more **restrictive queries** with the `WHERE` keyword:

In [7]:
# note that the triple quotes allow us to break lines
pd.read_sql("""SELECT * FROM albums 
                    WHERE Title = 'Big Ones'""", conn)

,AlbumId,Title,ArtistId
0,5,Big Ones,3


Legal operations are: 

 * `=` is.
 * `<>` is not
 * Numerical comparisons: `>, >=, <, <=`
 * `IN` allows you to pass a list of options.
 * `BETWEEN` allows you to define a range.
 * `LIKE` can be used with pattern matching, similar to regular expressions. 
 * `IS NULL` or `IS NOT NULL` test for empty values. 

The `is not` operator:

In [8]:
pd.read_sql("""SELECT * FROM albums 
                    WHERE Title <> 'Big Ones'""", conn).head(10)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,6,Jagged Little Pill,4
5,7,Facelift,5
6,8,Warner 25 Anos,6
7,9,Plays Metallica By Four Cellos,7
8,10,Audioslave,8
9,11,Out Of Exile,8


In [9]:
pd.read_sql("""Select * FROM invoices WHERE Total > 10""", conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
1,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
2,19,40,2009-03-14 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,13.86
3,26,19,2009-04-14 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,13.86
4,33,57,2009-05-15 00:00:00,"Calle Lira, 198",Santiago,None,Chile,None,13.86
5,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
6,47,15,2009-07-16 00:00:00,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,13.86
7,54,53,2009-08-16 00:00:00,113 Lupus St,London,None,United Kingdom,SW1V 3EN,13.86
8,61,32,2009-09-16 00:00:00,696 Osborne Street,Winnipeg,MB,Canada,R3L 2B9,13.86
9,68,11,2009-10-17 00:00:00,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,13.86


The `BETWEEN` keyword:

In [10]:
pd.read_sql("""Select * FROM invoices WHERE Total BETWEEN 15 and 20""", conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,88,57,2010-01-13 00:00:00,"Calle Lira, 198",Santiago,None,Chile,None,17.91
1,89,7,2010-01-18 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,18.86
2,103,24,2010-03-21 00:00:00,162 E Superior Street,Chicago,IL,USA,60611,15.86
3,201,25,2011-05-29 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,18.86
4,208,4,2011-06-29 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,15.86
5,306,5,2012-09-05 00:00:00,Klanova 9/506,Prague,None,Czech Republic,14700,16.86
6,313,43,2012-10-06 00:00:00,"68, Rue Jouvence",Dijon,None,France,21000,16.86


The `IN` keyword:

In [11]:
pd.read_sql("""Select * FROM invoices WHERE BillingCity in ('Chicago', 'London', 'Berlin')""", conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,11,52,2009-02-06 00:00:00,202 Hoxton Street,London,None,United Kingdom,N1 5LH,8.91
2,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
3,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
4,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
5,43,53,2009-07-06 00:00:00,113 Lupus St,London,None,United Kingdom,SW1V 3EN,1.98
6,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
7,54,53,2009-08-16 00:00:00,113 Lupus St,London,None,United Kingdom,SW1V 3EN,13.86
8,92,24,2010-02-08 00:00:00,162 E Superior Street,Chicago,IL,USA,60611,1.98
9,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91


Expressions with `LIKE`:

In [12]:
# starts with a C
pd.read_sql("""Select * FROM invoices WHERE BillingCity LIKE 'C%'""", conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98
1,26,19,2009-04-14 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,13.86
2,56,9,2009-09-06 00:00:00,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,1.98
3,79,9,2009-12-09 00:00:00,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,3.96
4,81,19,2009-12-13 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,8.91
5,92,24,2010-02-08 00:00:00,162 E Superior Street,Chicago,IL,USA,60611,1.98
6,101,9,2010-03-13 00:00:00,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,5.94
7,103,24,2010-03-21 00:00:00,162 E Superior Street,Chicago,IL,USA,60611,15.86
8,153,9,2010-11-01 00:00:00,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,0.99
9,158,24,2010-11-19 00:00:00,162 E Superior Street,Chicago,IL,USA,60611,8.91


In [13]:
# ends with rt
pd.read_sql("""Select * FROM invoices WHERE BillingCity LIKE '%rt'""", conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
3,67,2,2009-10-12 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,8.91
4,127,37,2010-07-13 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,1.98
5,138,37,2010-08-23 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,13.86
6,193,37,2011-04-23 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,14.91
7,196,2,2011-05-19 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
8,219,2,2011-08-21 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,3.96
9,241,2,2011-11-23 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,5.94


We can use `ORDER BY` to sort the output.

In [14]:
pd.read_sql("""SELECT * FROM albums 
                    ORDER BY title""", conn)

,AlbumId,Title,ArtistId
0,156,...And Justice For All,50
1,257,20th Century Masters - The Millennium Collecti...,179
2,296,"A Copland Celebration, Vol. I",230
3,94,A Matter of Life and Death,90
4,95,A Real Dead One,90
5,96,A Real Live One,90
6,285,A Soprano Inspired,219
7,139,A TempestadeTempestade Ou O Livro Dos Dias,99
8,203,A-Sides,132
9,160,Ace Of Spades,106


### Exercise 1: Simple Queries

1. List all the rows in the genres table.
2. List only the genre names (nothing else) in the table.
3. List the genre names ordered by name.
4. List the genre entries with IDs between 13 and 17.
5. List the genre entries that start with an S.
6. List the GenreIds of Rock, Jazz, and Reggae (in one query).

## Referencing and Renaming

Not very exciting on it's own, but we'll want to referene exact names and rename them soon. Here we're using a fully qualified name `tracks.Name` to refer to the name column in the tracks table, and then rename it as `TrackName`.

In [15]:
pd.read_sql("""SELECT tracks.Name as TrackName FROM tracks;""", conn).head()

,TrackName
0,For Those About To Rock (We Salute You)
1,Balls to the Wall
2,Fast As a Shark
3,Restless and Wild
4,Princess of the Dawn


## Joining Tables

There are many ways to aggregate and group by with SQL, and we won't be able to cover them in detail. We'll use a simple example for an `INNER JOIN` to resolve a foreign key relationship here.

We want to create a long table that contains both track titles and for each track the album title. Let's take a looks at the tracks table:

In [16]:
pd.read_sql("""SELECT * FROM tracks;""", conn).head()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


And the albums table:

In [17]:
pd.read_sql("""SELECT * FROM albums;""", conn).head(5)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


We can see that AlbumID appears in both tables. This is the relationship we care about:

![](albums_tracks_tables.jpg)

An INNER JOIN simply merges tables based on a primary key / foreign key relationship. 

Here is an example:

In [18]:
pd.read_sql("""SELECT
 trackid,
 tracks.name as Track,
 albums.title as Album
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid""", conn).head(30)

,TrackId,Track,Album
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,6,Put The Finger On You,For Those About To Rock We Salute You
2,7,Let's Get It Up,For Those About To Rock We Salute You
3,8,Inject The Venom,For Those About To Rock We Salute You
4,9,Snowballed,For Those About To Rock We Salute You
5,10,Evil Walks,For Those About To Rock We Salute You
6,11,C.O.D.,For Those About To Rock We Salute You
7,12,Breaking The Rules,For Those About To Rock We Salute You
8,13,Night Of The Long Knives,For Those About To Rock We Salute You
9,14,Spellbound,For Those About To Rock We Salute You


Let's take this statement apart:

```SQL
SELECT
 trackid,
 tracks.name as Track,
 albums.title as Album
```
Here, we chose what to select, and we also rename the columns for presentation purposes with the `as` statement. 

```SQL
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid;
```

This is where the magic happens: we say that the tracks and albums table should be joined on the `albums.albumid = tracks.albumid` relationship.

Using INNER JOIN, you can construct flat tables out of databases that you then can use in the data science process. SQL is of course much more powerful, you can do mathematical operations, group-by/aggregates, etc. However, we can do most of this on a dataframe also, but doing a join like that would be rather painful without the INNER JOIN SQL statement.

Next, we want to also add the artist name. We can just append another `INNER JOIN` statement.

In [19]:
pd.read_sql("""SELECT
 tracks.name as Track,
 albums.title as Album,
 artists.name as Artist 
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
INNER JOIN artists ON artists.artistid = albums.artistid""", conn).head(40)

,Track,Album,Artist
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC
1,Put The Finger On You,For Those About To Rock We Salute You,AC/DC
2,Let's Get It Up,For Those About To Rock We Salute You,AC/DC
3,Inject The Venom,For Those About To Rock We Salute You,AC/DC
4,Snowballed,For Those About To Rock We Salute You,AC/DC
5,Evil Walks,For Those About To Rock We Salute You,AC/DC
6,C.O.D.,For Those About To Rock We Salute You,AC/DC
7,Breaking The Rules,For Those About To Rock We Salute You,AC/DC
8,Night Of The Long Knives,For Those About To Rock We Salute You,AC/DC
9,Spellbound,For Those About To Rock We Salute You,AC/DC


Now we can combine that with a `WHERE` condition to get a more specific result. Here we're looking at all the Rolling Stones albums in this dataset. 

In [20]:
pd.read_sql("""SELECT
 tracks.name as Track,
 albums.title as Album,
 artists.name as Artist 
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
INNER JOIN artists ON artists.artistid = albums.artistid
WHERE artists.name = 'The Rolling Stones'""", conn).head(40)

,Track,Album,Artist
0,Time Is On My Side,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
1,Heart Of Stone,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
2,Play With Fire,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
3,Satisfaction,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
4,As Tears Go By,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
5,Get Off Of My Cloud,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
6,Mother's Little Helper,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
7,19th Nervous Breakdown,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
8,Paint It Black,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones
9,Under My Thumb,"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones


## Group By

We've seen group by before for dataframes, and we could just use pandas' group by since we're using a dataframe anyways. However, it's useful to understand how group by works in SQL as it can be much more efficient to just get the right data out in the first place. 

In [21]:
pd.read_sql("""SELECT
 albumid,
 COUNT(trackid) as '# Tracks'
FROM
 tracks
GROUP BY
 albumid;""", conn).head()

,AlbumId,# Tracks
0,1,10
1,2,1
2,3,3
3,4,8
4,5,15


These are accurate counts, but now we want to combined them with album titles. We first do the INNER JOIN and then the GROUP BY. 

In [22]:
pd.read_sql("""SELECT
 tracks.albumid,
 albums.title as Album,
 COUNT(tracks.trackid)
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
GROUP BY
 tracks.albumid""", conn).head(30)

,AlbumId,Album,COUNT(tracks.trackid)
0,1,For Those About To Rock We Salute You,10
1,2,Balls to the Wall,1
2,3,Restless and Wild,3
3,4,Let There Be Rock,8
4,5,Big Ones,15
5,6,Jagged Little Pill,13
6,7,Facelift,12
7,8,Warner 25 Anos,14
8,9,Plays Metallica By Four Cellos,8
9,10,Audioslave,14


We can combine this with filters, using `HAVING COUNT`.

In [23]:
pd.read_sql("""SELECT
 tracks.albumid,
 albums.title as Album,
 COUNT(tracks.trackid)
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
GROUP BY
 tracks.albumid
HAVING COUNT(tracks.trackid) > 15;""", conn).head(30)

,AlbumId,Album,COUNT(tracks.trackid)
0,18,Body Count,17
1,21,Prenda Minha,18
2,23,Minha Historia,34
3,24,Afrociberdelia,23
4,26,Acústico MTV [Live],17
5,33,Chill: Brazil (Disc 1),17
6,34,Chill: Brazil (Disc 2),17
7,36,Greatest Hits II,17
8,37,Greatest Kiss,20
9,39,International Superhits,21


Here are several other aggregation functions, `AVG, MAX` and `SUM`. We also do some simple math to convert miliseconds into minutes for the average: 

In [24]:
pd.read_sql("""SELECT
 tracks.albumid,
 albums.title as Album,
 COUNT(tracks.trackid),
 AVG(tracks.Milliseconds)/1000/60 as 'Average(minutes)',
 MAX(tracks.Milliseconds),
 SUM(tracks.Milliseconds)
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
GROUP BY
 tracks.albumid
ORDER BY 
  AVG(tracks.Milliseconds) DESC""", conn).head(30)

,AlbumId,Album,COUNT(tracks.trackid),Average(minutes),MAX(tracks.Milliseconds),SUM(tracks.Milliseconds)
0,253,"Battlestar Galactica (Classic), Season 1",24,48.759572,2960293,70213784
1,227,"Battlestar Galactica, Season 3",19,46.304422,5286953,52787041
2,229,"Lost, Season 3",26,45.298450,5088838,70665582
3,231,"Lost, Season 2",24,43.951133,2825166,63289631
4,226,Battlestar Galactica: The Story So Far,1,43.704167,2622250,2622250
5,228,"Heroes, Season 1",23,43.319035,2687103,59780268
6,230,"Lost, Season 1",25,43.236624,2631327,64854936
7,254,Aquaman,1,41.409450,2484567,2484567
8,261,"LOST, Season 4",17,38.694542,2612779,39468433
9,251,"The Office, Season 3",25,25.544730,2541875,38317095


## Exercise 2: Joining

1. Create a table that contains track names, genre name and genre ID for each track. Hint: the table is sorted by genres, look at the tail of the dataframe to make sure it works correctly.
2. Create a table that contains the counts of tracks in a genre by using the GenreID.
3. Create a table that contains the genre name and the count of tracks in that genre.
4. Sort the previous table by the count. Which are the biggest genres? Hint: the DESC keyword can be added at the end of the sorting expression.

## Security

We can use python variables to specify the columns. However, you typically shouldn't trust the content of the variables, especially in a user facing system. Imagine, you read in which attribute to query for from a website field where a user can specify a name. The user could use this to attack your SQL server using [SQL Injection](https://en.wikipedia.org/wiki/SQL_injection). For example, for this statement:

In [27]:
title = "Big Ones"
# executescript runs multiple sql statements. 
# This isn't very helpful for quering, but can be great for other operations.
c.executescript("""SELECT * FROM albums WHERE Title = '""" + title + "'")

Let's look at the tables in the database:

In [28]:
pd.read_sql("""SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
            """, conn)

,name
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoice_items
6,invoices
7,media_types
8,playlist_track
9,playlists


Now, if we were to read `title` from a user, and adversarial user could write: 

```SQL
"a';DROP TABLE invoice_items;"
```

This selects everything from table "a", then concludes the SQL command with `;`, and then executed the next SQL command `DROP TABLE invoice_items`, which – you guessed it – deletes the table invoice_items.

In [29]:
title = "a';DROP TABLE invoice_items;"
print("""SELECT * FROM albums WHERE Title = '""" + title + "'")
c.executescript("""SELECT * FROM albums WHERE Title = '""" + title + "'")

SELECT * FROM albums WHERE Title = 'a';DROP TABLE invoice_items;'


OperationalError: unrecognized token: "'"

This throws an error, but the table is gone:

In [30]:
pd.read_sql("""SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
            """, conn)

,name
0,albums
1,artists
2,customers
3,employees
4,genres
5,invoices
6,media_types
7,playlist_track
8,playlists
9,sqlite_sequence


Instead, you should use a ? for substitution, and pass in a tuple. This will make sure that no additional statements will be executed. 

In [31]:
title = ("a';DROP TABLE invoice_items;",)
# this will result in an error
c.executescript('SELECT * FROM albums WHERE title=?', title)

TypeError: function takes exactly 1 argument (2 given)

Here is a working example with safe code:

In [32]:
title = ("Big Ones",)
# this will result in an error
c.execute('SELECT * FROM albums WHERE title=?', title)
c.fetchone()

(5, 'Big Ones', 3)

Of course that's not all to say about making your code secure, but it's a start! 

![](exploits_of_a_mom.png)